# Kor LLM 모델 서빙 (SageMaker Python SDK)

### 참조: 
- Model 정보
    - beomi/KoAlpaca-Polyglot-12.8B
        - This model is a fine-tuned version of EleutherAI/polyglot-ko-12.8b on a KoAlpaca Dataset v1.1b
        - https://huggingface.co/beomi/KoAlpaca-Polyglot-12.8B
    - EleutherAI/polyglot-ko-12.8b
        - Polyglot-Ko-12.8B was trained for 167 billion tokens over 301,000 steps on 256 A100 GPUs with the GPT-NeoX framework. It was trained as an autoregressive language model, using cross-entropy loss to maximize the likelihood of predicting the next token.
        - License: Apache 2.0
        - https://huggingface.co/EleutherAI/polyglot-ko-12.8b
        
- Doc
    - Large model inference tutorials
        - https://docs.aws.amazon.com/sagemaker/latest/dg/large-model-inference-tutorials.html
    - Use DJL with the SageMaker Python SDK
        - https://sagemaker.readthedocs.io/en/stable/frameworks/djl/using_djl.html
        
- 블로그
    - https://aws.amazon.com/ko/blogs/machine-learning/deploy-large-models-on-amazon-sagemaker-using-djlserving-and-deepspeed-model-parallel-inference/
    - 코드
        - https://github.com/aws/amazon-sagemaker-examples/blob/main/inference/generativeai/deepspeed/GPT-J-6B_DJLServing_with_PySDK.ipynb

# 1. 기본 환경 설정

In [4]:
%load_ext autoreload
%autoreload 2

# src 폴더 경로 설정
import sys
sys.path.append('../common_code')

# 2. SageMaker endpoint 의 추론 도커 이미지 인 DLC image URL 가져오기
- We get DLC image URL for djl-deepspeed 0.21.0 and set SageMaker settings

In [5]:
import sagemaker, boto3
from sagemaker import image_uris


role = sagemaker.get_execution_role()  # execution role for the endpoint
session = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = session._region_name
bucket = session.default_bucket()  # bucket to house artifacts

img_uri = image_uris.retrieve(framework="djl-deepspeed", region=region, version="0.21.0")
img_uri

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


'763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.21.0-deepspeed0.8.3-cu117'

# 3. Set configuration

## 테스트 모델 지정

In [100]:
#serve_model = 'KoAlpaca-12-8B'
# serve_model = 'Polyglot-Kor-5-8B'
serve_model = 'Kullm-polyglot-12-8b-v2'

# 모델 패키징 할 파일 이름
model_artifact_name = f'{serve_model}.tar.gz'

# 모델 패키징 S3 위치
s3_location = f"s3://{bucket}/{serve_model}"

print("serve_model: ", serve_model)
print("model_artifact_name: ", model_artifact_name)
print("model packaging s3 location: ", s3_location)

serve_model:  Kullm-polyglot-12-8b-v2
model_artifact_name:  Kullm-polyglot-12-8b-v2.tar.gz
model packaging s3 location:  s3://sagemaker-us-east-1-419974056037/Kullm-polyglot-12-8b-v2


## 모델 다운로드

In [101]:
##수정중
# from huggingface_hub import snapshot_download
# from pathlib import Path
# import os

# # - This will download the model into the current directory where ever the jupyter notebook is running
# local_model_path = Path(".")
# local_model_path.mkdir(exist_ok=True)
# model_name = "beomi/KoAlpaca-Polyglot-12.8B"

# # Only download pytorch checkpoint files
# allow_patterns = ["*.json", "*.pt", "*.bin", "*.txt", "*.model"]

# # - Leverage the snapshot library to donload the model since the model is stored in repository using LFS
# model_download_path = snapshot_download(
#     repo_id=model_name,
#     cache_dir=local_model_path,
#     allow_patterns=allow_patterns,
# )
# model_download_path

## 로컬 모드 혹은 클라우드 모드

In [102]:
use_local_mode = False
# use_local_mode = True

if use_local_mode:
    instance_type = "local_gpu"
    from sagemaker.local import LocalSession
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
else:
    sagemaker_session = sagemaker.session.Session()
    instance_type = "ml.g5.12xlarge"
    
print("instance_type  :", instance_type)      

instance_type  : ml.g5.12xlarge


# 4. 모델 추론 코드 및 모델 설정 파일을 패키징
- `model.py` and `serving.properties`
- The code below creates the SageMaker model file (`model.tar.gz`) and upload it to S3. 

In [103]:
%%sh -s {serve_model} {model_artifact_name}
serve_model=$1
model_artifact_name=$2
echo $serve_model
echo $model_artifact_name

rm -rf $serve_model/.ipynb_checkpoints

tar -czvf $model_artifact_name $serve_model/

Kullm-polyglot-12-8b-v2
Kullm-polyglot-12-8b-v2.tar.gz
Kullm-polyglot-12-8b-v2/
Kullm-polyglot-12-8b-v2/serving.properties
Kullm-polyglot-12-8b-v2/model.py


## mode.tar.gz 및 pretrained model을 S3 업로드

In [104]:
model_tar_url = sagemaker.s3.S3Uploader.upload(model_artifact_name, s3_location)
print("model_tar_url: ", model_tar_url)

model_tar_url:  s3://sagemaker-us-east-1-419974056037/Kullm-polyglot-12-8b-v2/Kullm-polyglot-12-8b-v2.tar.gz


In [105]:
## 수정 중
# define a variable to contain the s3url of the location that has the model
#pretrained_model_location = f'{s3_location}/pretrained/' 
#print(f"Pretrained model will be uploaded to ---- > {pretrained_model_location}")

In [106]:
## 수정 중
# model_artifact = session.upload_data(path=model_download_path, key_prefix=f'{serve_model}/pretrained')
# print(f"Model uploaded to --- > {model_artifact}")
# print(f"We will set option.s3url={model_artifact}")

In [107]:
## 수정 중
# we plug in the appropriate model location into our `serving.properties` file based on the region in which this notebook is running
#jinja_env = jinja2.Environment()
#template = jinja_env.from_string(Path("mymodel/serving.properties").open().read())
#Path("mymodel/serving.properties").open("w").write(template.render(s3url=pretrained_model_location))
#!pygmentize mymodel/serving.properties | cat -n

# 5. SageMaker Model 생성

In [108]:
from sagemaker.model import Model
from sagemaker import image_uris, get_execution_role

def create_model(model_name, role, sagemaker_session, inference_image_uri, model_s3_url):
    model = Model(
        image_uri=inference_image_uri,
        model_data=model_s3_url,
        role=role,
        name=model_name,
        sagemaker_session=sagemaker_session,
    )
    return model

In [109]:
from datetime import datetime

time_stamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
model_name = f"{serve_model}-" + time_stamp

sm_model = create_model(model_name, role, sagemaker_session, img_uri, model_tar_url)

# 6. SageMaker Endpoint 생성
- 클라우드 배포시 약 8분 걸림. 10 이상 걸리면 무엇인가 문제 있음

In [110]:
from sagemaker import serializers, deserializers

def deploy_model(model, sagemaker_session, instance_type, _endpoint_name):
    model.deploy(
        initial_instance_count=1,
        instance_type=instance_type,
        endpoint_name=_endpoint_name
    )
    predictor = sagemaker.Predictor(
        endpoint_name=_endpoint_name,
        sagemaker_session=sagemaker_session,
        serializer=serializers.JSONSerializer(),
        deserializer=deserializers.JSONDeserializer()
    )
    return predictor

In [111]:
%%time
endpoint_name = f"{serve_model}-" + time_stamp
predictor = deploy_model(sm_model, sagemaker_session, instance_type, endpoint_name)

--------------!CPU times: user 129 ms, sys: 0 ns, total: 129 ms
Wall time: 7min 33s


# 7. 엔드포인트 추론 

In [7]:
from inference_lib import invoke_inference_DJ, Prompter

In [8]:
prompter = Prompter("kullm")

### options for generation
* **temperature**: Controls randomness in the model. Lower values will make the model more deterministic and higher values will make the model more random. Default value is 1.0.
* **max_new_tokens**: The maximum number of tokens to generate. Default value is 20, max value is 512.
* **repetition_penalty**: Controls the likelihood of repetition, defaults to null.
* **seed**: The seed to use for random generation, default is null.
* **stop**: A list of tokens to stop the generation. The generation will stop when one of the tokens is generated.
* **top_k**: The number of highest probability vocabulary tokens to keep for top-k-filtering. Default value is null, which disables top-k-filtering.
* **top_p**: The cumulative probability of parameter highest probability vocabulary tokens to keep for nucleus sampling, default to null
* **do_sample**: Whether or not to use sampling ; use greedy decoding otherwise. Default value is false.
* **best_of**: Generate best_of sequences and return the one if the highest token logprobs, default to null.
* **details**: Whether or not to return details about the generation. Default value is false.
* **return_full_text**: Whether or not to return the full text or only the generated part. Default value is false.
* **truncate**: Whether or not to truncate the input to the maximum length of the model. Default value is true.
* **typical_p**: The typical probability of a token. Default value is null.
* **watermark**: The watermark to use for the generation. Default value is false.

In [9]:
params = {
    "do_sample":False, 
    "max_new_tokens":128,
    "temperature":1.0,
    "top_k":0,
    "top_p":0.9,
    "return_full_text":False,
    "repetition_penalty":1.1,
    "presence_penalty":None,
    "eos_token_id":2,
}

## (1) 맥락 (Context) 없이 질문

In [10]:
q = "홈플러스 중계점은 몇시까지 장사해?"
c = ""#"홈플러스 영업시간은 오전 10시 부터 오후 12시까지 입니다."
prompt_wo_c = f"### 질문: {q}\n\n### 맥락: {c}\n\n### 답변:" if c else f"### 질문: {q}\n\n### 답변:" 
data = {
    "prompt": [prompt_wo_c,],
    "params": params
}
print("prompt_wo_c: \n", data)

prompt_wo_c: 
 {'prompt': ['### 질문: 홈플러스 중계점은 몇시까지 장사해?\n\n### 답변:'], 'params': {'do_sample': False, 'max_new_tokens': 128, 'temperature': 1.0, 'top_k': 0, 'top_p': 0.9, 'return_full_text': False, 'repetition_penalty': 1.1, 'presence_penalty': None, 'eos_token_id': 2}}


* **for kullm**

In [11]:
q = "홈플러스 중계점은 몇시까지 장사해?"
c = ""#"홈플러스 영업시간은 오전 10시 부터 오후 12시까지 입니다."
prompt_wo_c = prompter.generate_prompt(q, c)
data = {
    "prompt": [prompt_wo_c,],
    "params": params
}
print("prompt_wo_c: \n", data)

prompt_wo_c: 
 {'prompt': ['아래는 작업을 설명하는 명령어입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n홈플러스 중계점은 몇시까지 장사해?\n\n### 응답:\n'], 'params': {'do_sample': False, 'max_new_tokens': 128, 'temperature': 1.0, 'top_k': 0, 'top_p': 0.9, 'return_full_text': False, 'repetition_penalty': 1.1, 'presence_penalty': None, 'eos_token_id': 2}}


In [12]:
%%time 
invoke_inference_DJ(endpoint_name, data)

[
  [
    {
      "generated_text":"홈플러스 중계점의 영업시간은 매장마다 다를 수 있습니다. 홈플러스 중계점에 문의하여 영업 시간을 확인할 수 있습니다."
    }
  ]
]
CPU times: user 37.4 ms, sys: 8.53 ms, total: 45.9 ms
Wall time: 1.86 s


'[\n  [\n    {\n      "generated_text":"홈플러스 중계점의 영업시간은 매장마다 다를 수 있습니다. 홈플러스 중계점에 문의하여 영업 시간을 확인할 수 있습니다."\n    }\n  ]\n]'

## (2) 맥락 (Context) 가지고 질문

In [13]:
q = "홈플러스 중계점은 몇시까지 장사해?"
c = "홈플러스 영업시간은 오전 10시 부터 오후 10시까지 입니다. 홈플러스 매장 찾기(영업시간 확인)는 이 주소를 이용하세요:  http://corporate.homeplus.co.kr/Store.aspx?isA=%C1%F6%BF%B4%C7%B0%BF%AE%C0%C7%C1%F2%B5%B5%B4%F6 "
prompt_w_c = f"### 질문: {q}\n\n### 맥락: {c}\n\n### 답변:" if c else f"### 질문: {q}\n\n### 답변:" 
data = {
    "prompt": [prompt_w_c,],
    "params": params
}
print("prompt_w_c:\n", prompt_w_c)

prompt_w_c:
 ### 질문: 홈플러스 중계점은 몇시까지 장사해?

### 맥락: 홈플러스 영업시간은 오전 10시 부터 오후 10시까지 입니다. 홈플러스 매장 찾기(영업시간 확인)는 이 주소를 이용하세요:  http://corporate.homeplus.co.kr/Store.aspx?isA=%C1%F6%BF%B4%C7%B0%BF%AE%C0%C7%C1%F2%B5%B5%B4%F6 

### 답변:


* **for kullm**

In [16]:
q = "한글이 ?로 나오는데 어떻게 해야해?"
c = '''회원가입 페이지에서 자바스크립트로 닉네임 중복체크 유효성 검사를 구현했는데
사용자 입력 폼(회원가입 페이지)에서 팝업창으로 넘어간 한글 텍스트가 ???로 깨져서 나오는 문제가 발생했다.
해결방법:
톰캣 server.xml에 있는 <Connector/> 태그에 URIEncoding을 추가한다. JSP페이지도 인코딩 설정을 동일하게 해야 한다. 
한글이 깨지지 않으려면 보통 UTF-8이나 EUC-KR을 쓰는데 내 경우엔 인코딩 설정이 JSP페이지는 EUC-KR로 server.xml의 URIencoding은 UTF-8로 되어있어서 한글이 ???로 깨진 것이었다.'''
prompt_w_c = prompter.generate_prompt(q, c)
data = {
    "prompt": [prompt_w_c,],
    "params": params
}
print("prompt_w_c: \n", data)

prompt_w_c: 
 {'prompt': ['아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n한글이 ?로 나오는데 어떻게 해야해?\n\n### 입력:\n회원가입 페이지에서 자바스크립트로 닉네임 중복체크 유효성 검사를 구현했는데\n사용자 입력 폼(회원가입 페이지)에서 팝업창으로 넘어간 한글 텍스트가 ???로 깨져서 나오는 문제가 발생했다.\n해결방법:\n톰캣 server.xml에 있는 <Connector/> 태그에 URIEncoding을 추가한다. JSP페이지도 인코딩 설정을 동일하게 해야 한다. \n한글이 깨지지 않으려면 보통 UTF-8이나 EUC-KR을 쓰는데 내 경우엔 인코딩 설정이 JSP페이지는 EUC-KR로 server.xml의 URIencoding은 UTF-8로 되어있어서 한글이 ???로 깨진 것이었다.\n\n### 응답:\n'], 'params': {'do_sample': False, 'max_new_tokens': 128, 'temperature': 1.0, 'top_k': 0, 'top_p': 0.9, 'return_full_text': False, 'repetition_penalty': 1.1, 'presence_penalty': None, 'eos_token_id': 2}}


In [17]:
%%time 
invoke_inference_DJ(endpoint_name,  data)

[
  [
    {
      "generated_text":"URIEncoding을 JSP 페이지와 서버 XML 모두에서 UTF-8 또는 EUC-KR로 설정하여 한글이 깨지는지 확인할 수 있습니다. 이렇게 하려면 다음 단계를 따르세요:\n\n1. 서버 XML 파일(예: `server.xml`)에서 `connector` 태그에 URIEncoding을 추가합니다.\n2. JSP 페이지에서도 `server.xml`과 같은 방법으로 URIEncoding을 UTF-8 또는 EU"
    }
  ]
]
CPU times: user 12.3 ms, sys: 64 µs, total: 12.4 ms
Wall time: 7.59 s


'[\n  [\n    {\n      "generated_text":"URIEncoding을 JSP 페이지와 서버 XML 모두에서 UTF-8 또는 EUC-KR로 설정하여 한글이 깨지는지 확인할 수 있습니다. 이렇게 하려면 다음 단계를 따르세요:\\n\\n1. 서버 XML 파일(예: `server.xml`)에서 `connector` 태그에 URIEncoding을 추가합니다.\\n2. JSP 페이지에서도 `server.xml`과 같은 방법으로 URIEncoding을 UTF-8 또는 EU"\n    }\n  ]\n]'

# 7. [중요] 클린업 엔트포인트 

In [16]:
# delete endpoint
predictor.delete_model()
predictor.delete_endpoint()

# Trouble Shooting

3vep2qi5ar-algo-1-zk8i8 | INFO  ModelServer Model server stopped.

3vep2qi5ar-algo-1-zk8i8 | ERROR ModelServer Invalid configuration: Workflow KoAlpaca_12_8B is already registered.